### Chain

chain은 일련의 작동과 LLM 호출을 미리 정해둔 것이다. 체인을 활용하면 LLM을 서로 또는 다른 컴포넌트와 결합해야 하는 복잡한 애플리케이션을 쉽게 구축할 수 있다. 

랭체인은 시작하기 위한 네 가지 주요 유형의 체인을 제공한다.

In [3]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = openai.api_key

#### LLM Chain

가장 일반적인 유형의 체인이다. prompt template, LLM, output parser로 구성된다. 여기서 output parser는 언어 모델 응답을 구조화하는 데 도움이 되는 컴포넌트이다.

In [4]:
from langchain import PromptTemplate

template = """
sentence : {sentence}
translate to {language}
"""
prompt = PromptTemplate(
    template = template,
    input_variables = ['sentence','language']
)

이제 이를 LLMChain에 넣어 보자

In [6]:
from langchain.chains import LLMChain
from langchain_openai import OpenAI

llm = OpenAI(temperature = 0)
llm_chain = LLMChain(prompt = prompt, llm = llm)
llm_chain.predict(
    sentence = "지난 해 애플 주식을 10주 샀는데 20%가 상승했어요.",
    language = '영어'
)

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_63476/862837946.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt = prompt, llm = llm)


'Last year, I bought 10 shares of Apple stock and it rose by 20%.'

In [9]:
# LLMChain deprecation 해결

from langchain_openai import OpenAI
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnablePassthrough

template = """
sentence : {sentence}
translate to {language}
"""
prompt = PromptTemplate(
    template = template, 
    input_variables = ["sentence", "language"]
)

llm = OpenAI(temperature = 0)

output_parser = StrOutputParser()

chain = RunnableSequence(
    {
        "sentence": RunnablePassthrough(),
        "language": RunnablePassthrough()
    }
    | prompt
    | llm
    | output_parser
)

result = chain.invoke({
    'sentence' : "지난 해 애플 주식을 10주 샀는데 20%가 상승했어요.",
    'language' : '영어'
})
print(result)

Last year, I bought 10 shares of Apple stock and it rose by 20%.


#### RouterChain

특정 조건에 따라 입력 변수를 다른 체인으로 라우팅할 수 있는 체인 유형이다. 조건 값을 반환하는 함수나 표현식으로 지정할 수 있다. 조건이 충족되지 않는 경우 사용할 기본 체인을 지정할 수도 있다. 

예를 들어서 이 체인을 사용하여 여행 일정 계획이나 레스토랑 예약과 같은 다양한 유형의 요청을 처리할 수 있는 챗봇을 만들 수 있다. 이 목표를 달성하기 위해 사용자의 쿼리 유형에 따라 두 가지 프롬프트를 차별화할 수 있다.

In [11]:
portfolio_template = """
당신은 전문 투자 조언가입니다.
당신은 고객이 제시한 종목 중 상승 가능성이 가장 큰 종목 혹은 가장 성장가능성이 큰 종목을 찾도록 도와줍니다.
당신은 고객의 선호에 따라 2025년 기준으로 최선의 포트폴리오를 구성하도록 도움을 줍니다.

여기에 질문이 있습니다 :
{input}
"""

risk_manager_template = """
당신은 포트폴리오 리스크를 관리해주는 전문 리스크 매니저입니다.
당신은 고객이 보유중인 포트폴리오의 잠재적인 시장 위험 및 신용 위험, 운영 위험 등을 평가합니다.
여기서 고객이 얼마나 리스크를 감당할 수 있는지 고려해야 합니다.

여기 질문이 있습니다 :
{input}
"""

langchain 덕분에 사용자의 쿼리에 따라 다른 프롬프트를 활성화할 수 있는 체인을 구축할 수 있다. 체인이 두 가지 다른 사용자의 쿼리에 어떻게 반응하는지에 대한 샘플 출력을 볼 수 있다.

In [12]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

prompt_infos = [
    {
        "name": "포트폴리오 구성",
        "description": "주식 종목 포트폴리오 구성을 돕습니다",
        "prompt_template": portfolio_template,
    },
    {
        "name": "리스크 관리",
        "description": "고객의 포트폴리오 위험 관리를 도와줍니다",
        "prompt_template": risk_manager_template,
    },
]

In [13]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm = llm, output_key = "text")

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_63476/812518829.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(llm = llm, output_key = "text")
/Users/junghunlee/anaconda3/envs/langchain/lib/python3.11/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [14]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [15]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_63476/1641175374.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [17]:
print(
    chain.run('저는 현재 미국 NASDAQ에 상장되어 있는 SealSQ(LAES), Spectral AI(MDAI), IONQ(IONQ), Quantum Computing(QBUT), Rigetti Computing (RGTI)를 투자할까 고민중이에요.')
)

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_63476/1516597901.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run('저는 현재 미국 NASDAQ에 상장되어 있는 SealSQ(LAES), Spectral AI(MDAI), IONQ(IONQ), Quantum Computing(QBUT), Rigetti Computing (RGTI)를 투자할까 고민중이에요.')




> Entering new MultiPromptChain chain...
포트폴리오 구성: {'input': '저는 현재 미국 NASDAQ에 상장되어 있는 SealSQ(LAES), Spectral AI(MDAI), IONQ(IONQ), Quantum Computing(QBUT), Rigetti Computing (RGTI)를 투자할까 고민중이에요.'}
> Finished chain.
어떤 종목이 가장 성장 가능성이 높은지 조언해주실 수 있나요?

저는 이 종목들을 모두 분석해보고 고객의 투자 성향과 목표에 맞는 포트폴리오를 구성해드릴 수 있습니다. 하지만 각 종목의 성장 가능성을 판단하기 위해서는 더 많은 정보가 필요합니다. 종목의 재무상태, 시장 동향, 기술력 등을 종합적으로 고려하여 종목별 장단점을 분석하고, 고객의 투자 목표와 리스크 성향에 맞는 종목을 추천해드릴 수 있습니다. 따라서 이러한 종목들을 포함한 다양한 종목들을 분석한 후에 고객과 함께


#### Sequential Chain

여러 개의 체인을 순서대로 실행할 수 있는 체인 유형이다. 체인의 순서와 출력을 다음 체인에 전달하는 방법을 지정할 수 있다. 순차적 체인의 가장 단순한 모듈로, 기본적으로 한 체인의 출력을 다음 체인의 입력으로 사용한다. 그러나 더 복잡한 모듈을 사용하여 체인 간의 입출력을 보다 유연하게 설정할 수도 있다.

예를 들어, 주어진 주제에 대한 요약을 먼저 생성한 다음 이를 다른 언어로 번역하는 AI 시스템을 생각해 보자. 이를 위해 먼저 두 개의 체인을 생성한다.

In [19]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature = 0.7)
template = """
당신은 요약 전문가입니다. {text}에 관한 요약을 생성하세요.
본문 : 
"""
prompt_template = PromptTemplate(
    input_variables = ['text'],
    template = template
)
summary_chain = LLMChain(
    llm = llm,
    prompt = prompt_template
)

In [20]:
template = """
당신은 번역가입니다. 주어진 텍스트 입력을 {language}로 번역하세요.
번역 :
"""
prompt_template = PromptTemplate(
    input_variables = ['language'],
    template = template
)
translator_chain = LLMChain(
    llm = llm,
    prompt = prompt_template
)

이제 `SimpleSequentialChain` module을 사용해 이들을 결합해 보자

In [21]:
text_demo = """
Due to evolving legal landscape & changes in the framework of administrative law, Federal Reserve Board will soon seek public comment on significant changes to improve transparency of bank stress tests & reduce volatility of resulting capital requirements

In view of the evolving legal landscape, the Federal Reserve Board will soon seek public comment on significant changes to improve the transparency of its bank stress tests and to reduce the volatility of resulting capital buffer requirements.

The Board's stress test evaluates the resilience of large banks by estimating their losses, revenue, and capital levels under a hypothetical recession scenario that changes each year. Capital acts as a cushion to absorb losses and allows banks to continue lending to households and businesses even during a recession. Since its inception over 15 years ago, large banks in the stress test have more than doubled their capital levels, an increase of more than $1 trillion.

The Board intends to propose changes that include, but are not limited to: disclosing and seeking public comment on all of the models that determine the hypothetical losses and revenue of banks under stress; averaging results over two years to reduce the year-over-year changes in the capital requirements that result from the stress test; and ensuring that the public can comment on the hypothetical scenarios used annually for the test, before the scenarios are finalized. These proposed changes are not designed to materially affect overall capital requirements.

The framework of administrative law has changed significantly in recent years. The Board analyzed the current stress test in view of the evolving legal landscape and determined to modify the test in important respects to improve its resiliency.

The Board will continue its exploratory analysis, which assesses additional risks to the banking system in ways that are separate from the stress test. The analysis would be used to inform bank supervision and financial stability assessments. It will continue to be disclosed in aggregate and not affect bank capital requirements.

For the 2025 stress test, the Board plans to take immediate steps to reduce the volatility of the results and begin to improve model transparency. The Board intends to begin the public comment process on its comprehensive changes to the stress test during the early part of 2025.

For media inquiries, please email media@frb.gov or call 202-452-2955.
"""

In [22]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(
    chains = [summary_chain, translator_chain],
    verbose = True
)
translated_summary = overall_chain.run(text_demo)



> Entering new SimpleSequentialChain chain...

미국 연방준비제도이사회는 법적 환경의 변화와 행정법의 구조 변화를 고려하여, 은행 스트레스 테스트의 투명성을 개선하고 결과로 나오는 자본 요구량의 변동성을 줄이기 위한 중요한 변경안에 대한 대중의 의견을 곧 수렴할 예정입니다.

이사회의 스트레스 테스트는 매년 변화하는 가상의 경기 침체 시나리오를 기반으로 대규모 은행의 탄력성을 평가하며, 이를 통해 은행의 손실, 수입 및 자본 수준을 추정합니다. 자본은 손실을 흡수할 수 있는 공간을 제공하며, 은행이 경기 침체 기간 동안에도 가계
You are a translator. Given the input text, the Federal Reserve Board of Governors plans to soon incorporate public feedback on key changes aimed at improving the transparency of bank stress tests and reducing the volatility of capital requirements that result from changes in the legal and regulatory environment.

The stress tests, conducted by the Board each year, assess the resilience of large banks based on hypothetical economic downturn scenarios, estimating the banks' potential losses, income and capital levels. Capital provides a cushion to absorb losses and allows banks to continue lending to households even during an economic downturn.

> Finished chain.


#### Transformation Chain

일부 함수나 표현식을 사용해 입력 변수나 다른 체인의 출력을 변환할 수 있는 체인 유형이다. 입력 또는 출력을 인수로 받아 새 값을 반환하는 함수로 변환을 지정할 수 있으며, 체인의 출력 형식을 지정할 수도 있다.